In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import argparse
from numpy import linalg as LA
import scipy.stats as st
from scipy.optimize import minimize

In [1]:
## Write exponentially weighted covariance matrix function with input data and Lam
def ewCovar(x, lam):
    m, n = x.shape
    w = np.empty(m, dtype = float)
    ## Calculate the mean for each column
    xm = x.mean()
    ## Each x delete it stock return mean
    x = x - xm
    ## Calculate the weighted
    for i in range (0,m):
        w[i] = (1 - lam)*lam**(m-i-1)
    w = w / sum(w)
    ## Change w shape (60, ) to (60, 1)
    w = w[:, np.newaxis]
    ## make x to array for elementwise multiple
    x1 = np.array(x)
    res = np.multiply(w, x1)
    ## Transpose
    res = res.T
    ## matrix multiplication
    res = res @ x
    return res

In [3]:
## Test exponentially weighted convariance matrix function
## covar = ewCovar(df1,0.97)
## covar

In [4]:
#Write PCA function
from numpy.linalg import eigh
def PCA_pctExplained(a):
    m, n = a.shape
    egnvalues, egnvectors = eigh(a)
    ## reverse the value, sort from high to lower
    vals = sorted(egnvalues, reverse = True)
    ##Calculate the total Eigenvalues
    total = sum(vals)
    
    out = np.empty(n, dtype = float)
    s = 0.0
    for i in range(0, n):
        s += vals[i]
        out[i] = s/total
        
    return out

In [5]:
##Test PCA Function
##expl = PCA_pctExplained(covar)
##expl

In [6]:
## near_psd

import math
from numpy.linalg import eigh

def near_psd(A, epsilon=0):
    m,n = A.shape
    
    invSD = None
    out = np.array(A.copy()) 
    
    #calculate the correlation matrix if we got a covariance
    
    #if it gets passed a matrix that is not a correlation matrix (i.e. there is not 1 on the diagonals), 
    #then it converts it to a correlation matrix
    if np.sum(np.diag(out) == 1) != n:  
        invSD = np.diag(1 / np.sqrt(np.diag(out)))
        out = invSD * out * invSD
    
    
    #SVD, update the eigen value and scale
    eigval, eigvec = np.linalg.eigh(out)
    val = np.matrix(np.maximum(eigval,epsilon))
     
    vec = np.matrix(eigvec)
    T = 1/(np.multiply(vec,vec) * val.T)
    T = np.matrix(np.sqrt(np.diag(np.array(T).reshape((n)) )))
    l = np.diag(np.array(np.sqrt(val)).reshape((n)))
    B = T * vec * l
    out = B*B.T
    
    
    #Add back the variance
    if invSD != None: 
        invSD = np.diag(1 / np.diag(invSD))
        out = invSD * out * invSD
    
    return out

In [7]:
## cholesky assums PSD (chol_psd)
import math
from numpy.linalg import eigh

def chol_psd(root, a):
    n = a.shape[0]
    
    for j in range(n):
            s = 0.0
            if j > 0:
                s = root[j, :j] @ root[j, :j].T
            temp = a[j,j] - s
            if -1e-8 <= temp <= 0:
                temp = 0.0
            root[j,j] = np.sqrt(temp)
            if root[j,j] == 0.0:
                continue
            ir = 1.0 / root[j,j]
            for i in range((j + 1), n):
                s = root[i, :j] @ root[j, :j].T
                root[i, j] = (a[i,j] - s) * ir
    return root

In [10]:
##Higham's 2002 Helper Function
import math
from numpy.linalg import eigh


def _getAplus(A):
    vals, vecs =eigh(A)
    vals = np.matrix(np.diag(np.maximum(vals, 0)))
    return vecs*vals*vecs.T

def _getPS(A, W = None):
    W05 = np.matrix(np.sqrt(W))
    iW = W05.I
    return (iW @ _getAplus(W05 @ A @ W05) @ iW)

def _getPu(A, W = None):
    Aret = A.copy()
    for i in range(0, A.shape[0]):
        Aret[i,i] = 1.0
    return Aret

def _wgtNorm(A, W = None):
    W05 = np.sqrt(W)
    W05 = W05 @ A @ W05
    return (W05 * W05).sum()

In [11]:
## Higham's 2002 Nearest PSD funtion
import math
from numpy.linalg import eigh

def higham_nearestPSD(pc, W = None, epsilon = 1e-9, maxIter = 100, tol = 1e-9):
    n = pc.shape[0]
    if W == None:
        W = np.identity(n)

    deltaS = np.zeros((n,n))

    Yk = pc.copy()
    norml = 9223372036854775807
    i=1

    while i <= maxIter:
        # println("$i - $norml")
        Rk = Yk - deltaS
        #Ps Update
        Xk = _getPS(Rk, W)
        deltaS = Xk - Rk
        #Pu Update
        Yk = _getPu(Xk, W)
        #Get Norm
        norm = _wgtNorm(Yk - pc, W)
        #Smallest Eigenvalue
        vals, vecs = eigh(Yk)
        minEigVal = np.min(vals)

        # print("Yk: "); display(Yk)
        # print("Xk: "); display(Xk)
        # print("deltaS: "); display(deltaS)

        if ((norm - norml) < tol) and (minEigVal > -epsilon):
            # Norm converged and matrix is at least PSD
            break

        # println("$norml -> $norm")
        norml = norm
        i += 1
        
    if i < maxIter: 
        print("Converged in %d iterations.\n" % i)
    else:
        print("Convergence failed after %d iterations.\n" % (i-1))

    return Yk

In [12]:
## Correlation
def getCor(cov):
    cov_diag = np.diag(cov)
    invSD = np.diag(np.divide(1, np.sqrt(cov_diag)))
    cor = invSD * cov * invSD
    return cor

In [13]:
## Simulate Normal Function
def simulateNormal(cov,nsim):
    if(cov.shape[0] != len(cov)):
        raise exception("covariance matrix is not square")
        
    n = cov.shape[0]
    root = np.zeros(cov.shape)    
    root = chol_psd(root, cov)
    np.random.seed(1998)
    z = np.random.normal(size=(n, nsim))
    ans = root @ z
    return ans

In [14]:
## Simulate PCA Function
def simulate_pca(cov, nsim, target):

    vals, vecs = eigh(cov)
    
    tv = sum(vals)
    vals = vals[::-1]
    vecs = vecs[::-1]
    
    vals = np.maximum(vals, 0)
    
    cumm = np.cumsum(vals) / tv
    i=0
    for i in range(len(vals)):
        if cumm[i] < target:
            i += 1
        else:
            break
            
    vals = vals[0:i+1]
    vecs = vecs[:, :i+1]
    
    B = vecs @ np.diag(np.sqrt(vals))
    np.random.seed(1234)
    z = np.random.normal(size=(len(vals), nsim))
    return B @ z

In [15]:
## Covariance
def CombineV(var, cor):
    std = np.sqrt(var)
    n = len(var)
    cov = np.matrix(np.zeros((n,n)))
    for i in range(n):
        for j in range(n):
            cov[i,j] = cor[i,j] * std[i] * std[j]
            
    return cov

In [18]:
## Simulate returns assuming it follows a normal distribution
mu, sigma = 0, 0.02
n = 15000
ret = np.random.normal(mu, sigma, n-1)
pd.DataFrame(ret)
p0 = 100

In [19]:
## Get the best fit distribution
def get_best_distribution(data):
    dist_names = ["norm", "lognorm"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))     
        dist_results.append((dist_name, p))
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, params[best_dist]

In [20]:
## Return Calculation
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import argparse

def return_calculate(prices, method = None, dateColumn = "Date"):
    prices2 = np.matrix(prices.drop(columns = ['Date']))
    Vars = list(prices.columns.values)
    nVars = len(Vars)
    Vars.remove('Date')
    
    if nVars == len(Vars):
        print("DateColumn not in DataFrame")
        
    nVars = nVars - 1
    p = prices2
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.empty([n-1,m], dtype=float)
    for i in range(0, n-1):
        for j in range(0, m):
            p2[i,j] = p[i+1,j] / p[i,j]
    
    if method.upper() == "DISCRETE":
         p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = math.log(p2)
    else:
        print("Method must be in (\"LOG\",\"DISCRETE\")")
    
    out = prices[[dateColumn]]
    out = out.iloc[1: , :]
    for i in range(0, nVars):
        out.loc[:, Vars[i]] = p2[:, i]
    return(out)

In [21]:
## Example to use Return Function
## Return = return_calculate(df2, "DISCRETE", 'Date' )
## Return

In [22]:
# VaR with normal distribution
## import scipy.stats as st
## VaR_05 = st.norm.ppf(0.05, 0, STD)
## VaR_01 = st.norm.ppf(0.01, 0, STD)
## print(VaR_05, VaR_01)

In [24]:
## VaR with MLE
## Fit_return = st.t.fit(INTC)
## VaR_05 = st.t.ppf(0.05,Fit_return[0], Fit_return[1], Fit_return[2])
## VaR_01 = st.t.ppf(0.01, Fit_return[0], Fit_return[1], Fit_return[2])
## print(VaR_05, VaR_01)

In [25]:
## VaR using Historical Simulation
## Sorted_Return = sorted(INTC)
## Arr_Return = np.array(Sorted_Return)
## n_05 = 0.05 * Arr_Return.shape[0]
## n_01 = 0.01 * Arr_Return.shape[0]
## VaR_05 = Arr_Return[int(n_05)]
## VaR_01 = Arr_Return[int(n_01)]
## print(VaR_05, VaR_01)

In [26]:
## Empirical Distribution  of Return, in sample
## Arr_INTC = np.array(INTC)
## VaR_05 = np.quantile(Arr_INTC, 0.05)
## VaR_01 = np.quantile(Arr_INTC, 0.01)
## print(VaR_05, VaR_01)

In [27]:
## Empirical Distribution  of Return, out of sample
## intcout = pd.read_csv("INTC.csv")
## intcout = intcout[['Date','Close']]
## rets = return_calculate(intcout, method="DISCRETE", dateColumn="Date")
## rets.rename({"Close":"INTC"},axis=1, inplace=True)

## x2 = np.asarray(rets['INTC'])
## VaR_05_out = np.quantile(x2,0.05)
## VaR_01_out = np.quantile(x2,0.01)
## print(VaR_05_out, VaR_01_out)

In [28]:
## MC Normal VaR Function
def mc_calculate(holdings, names, sim_prices,n, PV):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices @ vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
    a=int(0.05*n)
    VaR = PV - pVals[a]
    
    return VaR

In [29]:
def pvals(holdings, names, sim_prices):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices@vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
        
    return pVals

In [30]:
def histVaR_calculate(holdings, names, sim_prices, PV, returns):
    vHoldings = np.array([holdings[s] for s in names])
    pVals = sim_prices@vHoldings
    pVals = np.array(pVals).ravel()
    pVals.sort()
    n = returns.shape[0]
    a=int(0.05*n)
    VaR = PV - pVals[a]
    return VaR

In [31]:
#function to calculate the portfolio value and delta.
def delta_pv_calculate(holdings, current_prices, names):
    PV = 0.0
    delta = np.empty(len(names), dtype=float)
    
    i=0
    for s in names:
        value = holdings[s] * current_prices[s]
        PV += value
        delta[i] = value
        i+=1
        
    return PV, delta

In [32]:
#function to calculate Delta Normal VaR

def deltaNormal(ret, delta, PV):   
    delta = delta.reshape(len(delta),1)
    sigma = np.cov(ret.T)
    eigen = LA.eigh(sigma)
    evals = eigen[0]
    p_sig = np.sqrt(delta.T@sigma@delta)
    VaR = -PV * st.norm.ppf(0.05) * p_sig
    return VaR

In [5]:
def VaR(data, alpha):
    data = sorted(data)
    ceil = int(np.ceil(len(data) * alpha))
    floor = int(np.floor(len(data) * alpha))
    return - (data[ceil] + data[floor]) / 2